# Import des modules

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import pearsonr

# Extrait SIRH

- Calcul des statistiques descriptives
- Objectif : faire ressortir des différences clés entre les employés

In [67]:
sirh = pd.read_csv('ExtraitSIRH.csv')
sirh.head(5)

,id_employee,age,genre,revenu_mensuel,statut_marital,departement,poste,nombre_experiences_precedentes,nombre_heures_travailless,annee_experience_totale,annees_dans_l_entreprise,annees_dans_le_poste_actuel
0,1,41,F,5993,CÃ©libataire,Commercial,Cadre Commercial,8,80,8,6,4
1,2,49,M,5130,MariÃ©(e),Consulting,Assistant de Direction,1,80,10,10,7
2,4,37,M,2090,CÃ©libataire,Consulting,Consultant,6,80,7,0,0
3,5,33,F,2909,MariÃ©(e),Consulting,Assistant de Direction,1,80,8,8,7
4,7,27,M,3468,MariÃ©(e),Consulting,Consultant,9,80,6,2,2


# Extrait Evaluation Performance

- Calcul des statistiques descriptives
- Objectif : faire ressortir des différences clés entre les employés

In [68]:
eval_perf = pd.read_csv('ExtraitEvaluationsPerformance.csv')
eval_perf.head(5)

,satisfaction_employee_environnement,note_evaluation_precedente,niveau_hierarchique_poste,satisfaction_employee_nature_travail,satisfaction_employee_equipe,satisfaction_employee_equilibre_pro_perso,eval_number,note_evaluation_actuelle,heure_supplementaires,augementation_salaire_precedente
0,2,3,2,4,1,1,E_1,3,Oui,11 %
1,3,2,2,2,4,3,E_2,4,Non,23 %
2,4,2,1,3,2,3,E_4,3,Oui,15 %
3,4,3,1,3,3,3,E_5,3,Oui,11 %
4,1,3,1,2,4,3,E_7,3,Non,12 %


# Extrait Sondage

- Calcul des statistiques descriptives
- Objectif : faire ressortir des différences clés entre les employés

In [69]:
sondage = pd.read_csv('ExtraitSondage.csv')
sondage.head(5)

,a_quitte_l_entreprise,nombre_participation_pee,nb_formations_suivies,nombre_employee_sous_responsabilite,code_sondage,distance_domicile_travail,niveau_education,domaine_etude,ayant_enfants,frequence_deplacement,annees_depuis_la_derniere_promotion,annes_sous_responsable_actuel
0,Oui,0,0,1,1,1,2,Infra & Cloud,Y,Occasionnel,0,5
1,Non,1,3,1,2,8,1,Infra & Cloud,Y,Frequent,1,7
2,Oui,0,3,1,4,2,2,Autre,Y,Occasionnel,0,0
3,Non,0,3,1,5,3,4,Infra & Cloud,Y,Frequent,3,0
4,Non,1,3,1,7,2,1,Transformation Digitale,Y,Occasionnel,2,2


# Création du fichier central

- Identifier les colonnes permettant de réaliser des jointures entre les 3 fichiers :
    - ExtraitSIRH = id_employee
    - ExtraitEvaluationsPerformance = eval_number
    - ExtraitSondage = code_sondage


In [70]:
###########
# PANDAS
##########

# Modifier des donnes de la colonne "eval_number" de "eval_perf"
eval_perf['eval_number'] = eval_perf['eval_number'].str.replace('E_', '', regex=False).astype(int)
#print(eval_perf['eval_number'])

fc_temp = pd.merge(sirh, eval_perf, left_on="id_employee", right_on="eval_number")
fc = pd.merge(fc_temp, sondage, left_on="id_employee", right_on="code_sondage")
fc.head(5)

##########
# SQL
##########


,id_employee,age,genre,revenu_mensuel,statut_marital,departement,poste,nombre_experiences_precedentes,nombre_heures_travailless,annee_experience_totale,...,nb_formations_suivies,nombre_employee_sous_responsabilite,code_sondage,distance_domicile_travail,niveau_education,domaine_etude,ayant_enfants,frequence_deplacement,annees_depuis_la_derniere_promotion,annes_sous_responsable_actuel
0,1,41,F,5993,CÃ©libataire,Commercial,Cadre Commercial,8,80,8,...,0,1,1,1,2,Infra & Cloud,Y,Occasionnel,0,5
1,2,49,M,5130,MariÃ©(e),Consulting,Assistant de Direction,1,80,10,...,3,1,2,8,1,Infra & Cloud,Y,Frequent,1,7
2,4,37,M,2090,CÃ©libataire,Consulting,Consultant,6,80,7,...,3,1,4,2,2,Autre,Y,Occasionnel,0,0
3,5,33,F,2909,MariÃ©(e),Consulting,Assistant de Direction,1,80,8,...,3,1,5,3,4,Infra & Cloud,Y,Frequent,3,0
4,7,27,M,3468,MariÃ©(e),Consulting,Consultant,9,80,6,...,3,1,7,2,1,Transformation Digitale,Y,Occasionnel,2,2


# Colonnes à traiter
- statut_marital, poste => remplacer les caracteres illisibles
- nombre_heures_travailles, nombre_employee_sous_responsabilite, ayant_enfants => A supprimer car données identiques

In [71]:
print(f"Colonnes avec des données uniques: {fc.columns[fc.nunique() == 1].tolist()}")
fc.drop((['nombre_heures_travailless','nombre_employee_sous_responsabilite','ayant_enfants']), axis=1, inplace=True)

print(fc.groupby(['statut_marital']).size().sort_values(ascending=False))
print(fc.groupby(['poste']).size().sort_values(ascending=False))

for col in fc.select_dtypes(include='object').columns:
    fc[col] = fc[col].str.replace('Ã©', 'e', regex=False)

print(fc.groupby(['statut_marital']).size().sort_values(ascending=False))
print(fc.groupby(['poste']).size().sort_values(ascending=False))


Colonnes avec des données uniques: ['nombre_heures_travailless', 'nombre_employee_sous_responsabilite', 'ayant_enfants']
statut_marital
MariÃ©(e)       673
CÃ©libataire    470
DivorcÃ©(e)     327
dtype: int64
poste
Cadre Commercial            326
Assistant de Direction      292
Consultant                  259
Tech Lead                   145
Manager                     131
Senior Manager              102
ReprÃ©sentant Commercial     83
Directeur Technique          80
Ressources Humaines          52
dtype: int64
statut_marital
Marie(e)       673
Celibataire    470
Divorce(e)     327
dtype: int64
poste
Cadre Commercial           326
Assistant de Direction     292
Consultant                 259
Tech Lead                  145
Manager                    131
Senior Manager             102
Representant Commercial     83
Directeur Technique         80
Ressources Humaines         52
dtype: int64
